In [ ]:
from tensorflow.keras.datasets import fashion_mnist
from tensorflow import keras
import time

start_time = time.perf_counter()

# Charger l'ensemble de données Fashion MNIST
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()


# Listes d'hyperparamètres
nombre_epochs = 50
taux_apprentissage = [0.01, 0.001, 0.0001]
batch_sizes = [64, 128, 256]
architectures_couches = [[64], [128], [64, 64], [128, 64]]
taux_dropout = [0.2, 0.3, 0.5]
fonctions_perte = ['sparse_categorical_crossentropy', 'categorical_crossentropy', 'binary_crossentropy', 'mean_squared_error', 'mean_absolute_error', 'kullback_leibler_divergence', 'poisson', 'cosine_similarity']
optimiseurs = [keras.optimizers.Adam, keras.optimizers.SGD, keras.optimizers.RMSprop, keras.optimizers.Adagrad, keras.optimizers.Adadelta, keras.optimizers.Nadam, keras.optimizers.Ftrl]
fonctions_activation = ['relu', 'sigmoid', 'tanh', 'softmax']

meilleurs_resultats = []
meilleure_accuracy = 0

for lr in taux_apprentissage:
    for batch_size in batch_sizes:
        for architecture in architectures_couches:
            for dropout in taux_dropout:
                for loss_function in fonctions_perte:
                    for Optimizer in optimiseurs:
                        for activation_function in fonctions_activation:
                            # Construction du modèle
                            net = keras.models.Sequential()
                            net.add(keras.layers.Flatten(input_shape=(28, 28)))
                            for neurones in architecture:
                                net.add(keras.layers.Dense(neurones, activation=activation_function))
                                net.add(keras.layers.Dropout(dropout))
                            net.add(keras.layers.Dense(10, activation='softmax'))  # softmax pour classification

                            # Compilation du modèle
                            optimizer = Optimizer(learning_rate=lr)
                            net.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

                            # Callback pour l'arrêt prématuré
                            early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

                            # Entraînement du modèle
                            history = net.fit(X_train, y_train, epochs=nombre_epochs, batch_size=batch_size, validation_split=0.2, callbacks=[early_stopping_cb], verbose=0)

                            # Évaluer la performance du modèle
                            current_accuracy = max(history.history['val_accuracy'])

                            # Comparer et stocker le meilleur résultat
                            if current_accuracy > meilleure_accuracy:
                                meilleure_accuracy = current_accuracy
                                meilleurs_resultats = [
                                    ('taux_apprentissage', lr),
                                    ('batch_size', batch_size),
                                    ('architecture', architecture),
                                    ('dropout', dropout),
                                    ('fonction_perte', loss_function),
                                    ('optimiseur', Optimizer.__name__),
                                    ('fonction_activation', activation_function),
                                    ('accuracy', meilleure_accuracy)
                                ]
                            # Afficher les meilleurs hyperparamètres
                            print("Meilleurs hyperparamètres : ", meilleurs_resultats)

# Afficher les meilleurs hyperparamètres
print("Meilleurs hyperparamètres : ", meilleurs_resultats)
nd_time = time.perf_counter()
duration = end_time - start_time
print(f"Temps d'exécution du script: {duration} secondes")
